In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pickle
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import VotingRegressor
import itertools

In [2]:
df1=pd.read_csv("tjk_data_processed.csv",index_col = [0])

c:\users\cagat\appdata\local\programs\python\python39\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1108882 entries, 0 to 1139073
Data columns (total 78 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   age                         1108882 non-null  int64  
 1   weight                      1108882 non-null  float64
 2   city                        1108882 non-null  object 
 3   ada                         1108882 non-null  int64  
 4   ank                         1108882 non-null  int64  
 5   bur                         1108882 non-null  int64  
 6   diy                         1108882 non-null  int64  
 7   ela                         1108882 non-null  int64  
 8   ist                         1108882 non-null  int64  
 9   izm                         1108882 non-null  int64  
 10  koc                         1108882 non-null  int64  
 11  urf                         1108882 non-null  int64  
 12  short                       1108882 non-null  int64  
 1

In [4]:
distance="short"
surface="Dirt"

# Distance masked dataframe (picks only data in same distance category)
df_mask_distance = df1[df1["distance_cat"]==distance]

# Surface masked dataframe (picks only data in same surface)
df_mask_surface = df1[df1["surface"]==surface]

# Both same surface and same distance category masked dataframe (Will be used as train set)
df_double_mask = df_mask_distance[df_mask_distance["surface"]==surface]

#Creates a dictionary with required column names
test_set_dict_k1={"thoroughbred_id":[],"jockey_wa":[],"thoroughbred_surface_wa":[],"mother_surface_wa":[],
                   "father_surface_wa":[],"thoroughbred_distance_wa":[],"father_distance_wa":[],"mother_distance_wa":[],
                   "mother_general_wa":[],"father_general_wa":[]} # Wa means Weighted Average

# Function that creates dataframe from dictionaries
def train_set_to_x_y(df):
    df_train_total = df[["jockey_wa","thoroughbred_surface_wa","mother_surface_wa","father_surface_wa",
                         "thoroughbred_distance_wa","father_distance_wa","mother_distance_wa","mother_general_wa",
                         "father_general_wa","centiseconds"]]
    y_train=df_train_total["centiseconds"]
    X_train=df_train_total.copy()
    del X_train["centiseconds"]
    return X_train,y_train,df_train_total

X_train,y_train,df_train_total = train_set_to_x_y(df_double_mask)

In [25]:
df_train_total

,jockey_wa,thoroughbred_surface_wa,mother_surface_wa,father_surface_wa,thoroughbred_distance_wa,father_distance_wa,mother_distance_wa,mother_general_wa,father_general_wa,centiseconds
0,0.215971,0.378049,0.239698,0.216485,0.452941,0.221374,0.225455,0.197015,0.196410,9405
1,0.234601,0.186364,0.242809,0.213500,0.200000,0.190651,0.202105,0.229280,0.200867,9617
2,0.135422,0.390000,0.175978,0.211000,0.200000,0.186903,0.159664,0.162583,0.185879,9749
3,0.237233,0.108571,0.102222,0.115169,0.133333,0.121289,0.078125,0.097872,0.129344,10049
12,0.263259,0.267000,0.267000,0.126294,0.302326,0.116803,0.302326,0.231034,0.134732,10294
...,...,...,...,...,...,...,...,...,...,...
1138972,0.187292,0.222222,0.271429,0.234439,0.182353,0.232649,0.238889,0.239394,0.224926,10701
1138973,0.187944,0.000000,0.465116,0.183257,0.000000,0.201343,0.385106,0.406024,0.193552,10720
1138974,0.156719,0.060000,0.060000,0.197485,0.060000,0.187191,0.060000,0.042857,0.181493,10746
1138975,0.112419,0.172727,0.198214,0.195779,0.214286,0.204669,0.279747,0.231405,0.198864,10785


In [26]:
corr_matrix = df_train_total.corr()

In [27]:
corr_matrix['centiseconds'].sort_values(ascending=False)

centiseconds                1.000000
father_surface_wa          -0.100659
mother_surface_wa          -0.102923
father_general_wa          -0.109068
mother_general_wa          -0.112858
father_distance_wa         -0.124348
mother_distance_wa         -0.127822
thoroughbred_surface_wa    -0.133675
jockey_wa                  -0.140096
thoroughbred_distance_wa   -0.152720
Name: centiseconds, dtype: float64

# Tuning Hyperparameters

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [50, 100, 150], 'learning_rate': [0.1, 0.2, 0.3], 'max_depth': [1, 2, 3]}
]
grad = GradientBoostingRegressor()
grid_search = GridSearchCV(grad, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid=[{'learning_rate': [0.1, 0.2, 0.3],
                          'max_depth': [1, 2, 3],
                          'n_estimators': [50, 100, 150]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [6]:
grid_search.best_params_

{'learning_rate': 0.3, 'max_depth': 3, 'n_estimators': 150}

In [8]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [150,200], 'learning_rate': [0.3,0.4], 'max_depth': [3,4]}
]
grad = GradientBoostingRegressor()
grid_search = GridSearchCV(grad, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
grid_search.fit(X_train, y_train)

Wall time: 1h 7min 30s


GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid=[{'learning_rate': [0.3, 0.4], 'max_depth': [3, 4],
                          'n_estimators': [150, 200]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [9]:
grid_search.best_params_

{'learning_rate': 0.4, 'max_depth': 4, 'n_estimators': 200}

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [200,250], 'learning_rate': [0.4,0.5], 'max_depth': [4,5]}
]
grad = GradientBoostingRegressor()
grid_search = GridSearchCV(grad, param_grid, cv=5,
scoring='neg_mean_squared_error',
verbose = 3,
return_train_score=True)
grid_search.fit(X_train, y_train)

In [11]:
grid_search.best_params_

{'learning_rate': 0.4, 'max_depth': 4, 'n_estimators': 200}

In [12]:
%%time 
reg1 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.4,max_depth=4, loss='ls')
reg1.fit(X_train, y_train)

Wall time: 2min 56s


GradientBoostingRegressor(learning_rate=0.4, max_depth=4, n_estimators=200)

In [13]:
param_grid = [
{'alpha': [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
]
lr = Lasso()
grid_search = GridSearchCV(lr, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Lasso(),
             param_grid=[{'alpha': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,
                                    0.8, 0.9, 1]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [14]:
grid_search.best_params_

{'alpha': 0.4}

In [9]:
param_grid = [
{'alpha': [1,2,4,10,40,80,85,90,91,100,200,300,400,500]}
]
rr = Ridge()
grid_search = GridSearchCV(rr, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True,
verbose = 3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
[CV 1/5] END alpha=1;, score=(train=-934400.690, test=-1421569.876) total time=   0.0s
[CV 2/5] END alpha=1;, score=(train=-1038327.385, test=-983532.839) total time=   0.0s
[CV 3/5] END alpha=1;, score=(train=-1047121.145, test=-950777.650) total time=   0.0s
[CV 4/5] END alpha=1;, score=(train=-1053294.066, test=-922149.864) total time=   0.0s
[CV 5/5] END alpha=1;, score=(train=-1048860.668, test=-937868.519) total time=   0.0s
[CV 1/5] END alpha=2;, score=(train=-934403.798, test=-1421301.110) total time=   0.0s
[CV 2/5] END alpha=2;, score=(train=-1038328.165, test=-983521.736) total time=   0.0s
[CV 3/5] END alpha=2;, score=(train=-1047122.372, test=-950755.342) total time=   0.0s
[CV 4/5] END alpha=2;, score=(train=-1053294.804, test=-922189.599) total time=   0.0s
[CV 5/5] END alpha=2;, score=(train=-1048861.274, test=-937890.361) total time=   0.0s
[CV 1/5] END alpha=4;, score=(train=-934414.935, test=-1420805.519) t

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid=[{'alpha': [1, 2, 4, 10, 40, 80, 85, 90, 91, 100, 200,
                                    300, 400, 500]}],
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=3)

In [7]:
grid_search.best_params_

{'alpha': 90}

In [42]:
%%time
param_grid = [
{'n_neighbors': [5,10,20]}
]
knnr = KNeighborsRegressor()
grid_search = GridSearchCV(knnr, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True,
verbose=3)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END n_neighbors=5;, score=(train=-593978.765, test=-1624237.378) total time=  16.6s
[CV 2/5] END n_neighbors=5;, score=(train=-648694.889, test=-1234788.337) total time=  15.5s
[CV 3/5] END n_neighbors=5;, score=(train=-652880.976, test=-1178707.077) total time=  14.3s
[CV 4/5] END n_neighbors=5;, score=(train=-662288.766, test=-1105499.757) total time=  13.7s
[CV 5/5] END n_neighbors=5;, score=(train=-656702.007, test=-1161875.960) total time=  13.3s
[CV 1/5] END n_neighbors=10;, score=(train=-702336.857, test=-1520332.924) total time=  20.6s
[CV 2/5] END n_neighbors=10;, score=(train=-772755.302, test=-1123989.836) total time=  18.6s
[CV 3/5] END n_neighbors=10;, score=(train=-781710.403, test=-1077091.195) total time=  17.6s
[CV 4/5] END n_neighbors=10;, score=(train=-791826.228, test=-1012531.267) total time=  18.5s
[CV 5/5] END n_neighbors=10;, score=(train=-784127.963, test=-1063818.380) total time=  17.4s
[CV 1

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid=[{'n_neighbors': [5, 10, 20]}], return_train_score=True,
             scoring='neg_mean_squared_error', verbose=3)

In [43]:
grid_search.best_params_

{'n_neighbors': 20}

In [ ]:
%%time
param_grid = [
{'n_neighbors': [30,40]}
]
knnr = KNeighborsRegressor()
grid_search = GridSearchCV(knnr, param_grid, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True,
verbose=3)
grid_search.fit(X_train, y_train)

# FINAL TRAIN

In [34]:
model_dict={}
distance_list=["short","middle","long"]
surface_list=["Turf","Dirt"]
l = [distance_list,surface_list]
for i in list(itertools.product(*l)):
    temp_model_list=[]
    distance=i[0]
    surface=i[1]
    
    # Distance masked dataframe (picks only data in same distance category)
    df_mask_distance = df1[df1["distance_cat"]==distance]

    # Surface masked dataframe (picks only data in same surface)
    df_mask_surface = df1[df1["surface"]==surface]

    # Both same surface and same distance category masked dataframe (Will be used as train set)
    df_double_mask = df_mask_distance[df_mask_distance["surface"]==surface]

    #Creates a dictionary with required column names
    test_set_dict_k1={"thoroughbred_id":[],"jockey_wa":[],"thoroughbred_surface_wa":[],"mother_surface_wa":[],
                       "father_surface_wa":[],"thoroughbred_distance_wa":[],"father_distance_wa":[],"mother_distance_wa":[],
                       "mother_general_wa":[],"father_general_wa":[]} # Wa means Weighted Average

    # Function that creates dataframe from dictionaries
    def train_set_to_x_y(df):
        df_train_total = df[["jockey_wa","thoroughbred_surface_wa","mother_surface_wa","father_surface_wa",
                             "thoroughbred_distance_wa","father_distance_wa","mother_distance_wa","mother_general_wa",
                             "father_general_wa","centiseconds"]]
        y_train=df_train_total["centiseconds"]
        X_train=df_train_total
        del X_train["centiseconds"]
        return X_train,y_train

    X_train,y_train = train_set_to_x_y(df_double_mask)

    name = str(distance)+"_"+str(surface.lower())
    
    #Fits train sets to models and puts into dict
    
    reg1 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.4,max_depth=4, loss='ls')
    reg2 = Lasso(alpha=0.4)
    reg3 = Ridge(alpha=90)
    reg4 = KNeighborsRegressor(n_neighbors=30)

    ereg = VotingRegressor(estimators=[('gb', reg1), ('lr', reg2),('rr',reg3),('knnr',reg4)])
    ereg = ereg.fit(X_train, y_train)
    temp_model_list.append(ereg)
    print(name+' voting done')

    gbr = GradientBoostingRegressor(n_estimators=200, learning_rate=0.4,max_depth=4, loss='ls')
    gbr.fit(X_train, y_train)
    temp_model_list.append(gbr)
    print(name+' gbr done')

    knnr = KNeighborsRegressor(n_neighbors=30)
    knnr.fit(X_train, y_train)
    temp_model_list.append(knnr)
    print(name+' knn done')

    lr = Lasso(alpha=0.4)
    lr.fit(X_train, y_train)
    temp_model_list.append(lr)
    print(name+' lasso done')

    rr = Ridge(alpha=90)
    rr.fit(X_train, y_train)
    temp_model_list.append(rr)
    print(name+' ridge done')
    
    model_dict[name]=temp_model_list

#Saves created dictionary
with open("models.pkl", "wb") as f:
    pickle.dump(model_dict, f)


short_turf voting done
short_turf gbr done
short_turf knn done
short_turf lasso done
short_turf ridge done
short_dirt voting done
short_dirt gbr done
short_dirt knn done
short_dirt lasso done
short_dirt ridge done
middle_turf voting done
middle_turf gbr done
middle_turf knn done
middle_turf lasso done
middle_turf ridge done
middle_dirt voting done
middle_dirt gbr done
middle_dirt knn done
middle_dirt lasso done
middle_dirt ridge done
long_turf voting done
long_turf gbr done
long_turf knn done
long_turf lasso done
long_turf ridge done
long_dirt voting done
long_dirt gbr done
long_dirt knn done
long_dirt lasso done
long_dirt ridge done
